# **Proyek Analisis Data: Bike-sharing-dataset**
 - **Nama:**
 - **Email:**
 - **ID Dicoding:**

## **Menentukan Pertanyaan Bisnis**

- Pertanyaan 1 : Apakah ada perbedaan jumlah penyewaan sepeda antara hari kerja (workingday) dan akhir pekan/libur?
- Pertanyaan 2 : Bagaimana faktor cuaca (weathersit) memengaruhi jumlah penyewaan sepeda harian?
- Pertanyaan 3 : Berapa banyak Total Penyewaan Setiap Disetiap Bulannya?


## **Import Semua Packages/Library yang Digunakan**

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import urllib
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

## **1. Data Wrangling**

### 1.1 Gathering Data

In [ ]:
day_df = pd.read_csv("data/day.csv")
day_df.head()

In [ ]:
data['day'].head()

menampilkan informasi 5 data teratas dari dataset **day**

---

In [ ]:
data['hour'].head()

menampilkan informasi 5 data teratas dari dataset **hour**

---

**Insight:**
- Bisa kita lihat bahwasanya dengan *.head()* akan menampilkan 5 data teratas
- Terdapat beberapa colomn seperti *dteday, season, casual, registered*, dsb. yang bisa kita gunakan untuk menganalisis selanjutnya

---

### 1.2 Assessing Data

- 1.2.1 Memeriksa semua tipe data

In [ ]:
for df_name, df in data.items():
    print(f"\n{df_name} Information:")
    print(data[df_name].info())

Semua nilai telah terisi lengkap, tidak terdapat nilai kosong (*null value*) pada tabel **day** maupun **hour**

---

- 1.2.2 Memeriksa data duplikat

In [ ]:
duplicate_counts = {}
total_sample_counts = {}

for df_name, df in data.items():
    duplicate_counts[df_name] = df.duplicated().sum()
    total_sample_counts[df_name] = len(df)

duplicate_counts_df = pd.DataFrame.from_dict(duplicate_counts, orient='index', columns=['Jumlah Duplikat'])
total_sample_counts_df = pd.DataFrame.from_dict(total_sample_counts, orient='index', columns=['Total Jumlah Sampel'])

pd.concat([total_sample_counts_df, duplicate_counts_df], axis=1)

Kedua datasetnya tidak ada yang duplikat

---

- 1.2.3 Memeriksa Nilai yang Hilang

In [ ]:
null_counts = {}

for df_name, df in data.items():
    null_counts[df_name] = df.isna().sum()
    print(df_name, '\n', null_counts[df_name], '\n')

Tidak ada nilai yang hilang *(missing value)* di kedua dataset

---

In [ ]:
day.describe()

Ini adalah parameter statistik dari kolom numerik yang terdapat didalam data **day**

---

In [ ]:
hour.describe()

Ini adalah parameter statistik dari kolom numerik yang terdapat didalam data **hour**

---

**Insight:**
- Memeriksa dan menelusuri data yang hilang maupun kosong terhadap dataset **day** dan **hour**
- Semua data terisi dan tidak hilang
- Tidak memiliki data yang terduplikasi

---

### 1.3 Cleaning Data

saya memeriksa datanya terlebih dahulu;

In [ ]:
print(day.isnull().sum())

Disini saya memeriksa kembali agar tabel **day** tidak ada nilai yang kosong, maka dari itu tidak perlu ada nilai yang harus diisi

---

In [ ]:
print(hour.isnull().sum())

Disini saya memeriksa kembali agar tabel **hour**  tidak ada nilai yang kosong, maka dari itu tidak perlu ada nilai yang harus diisi

---

1.3.1 Ubah Tipe Data

In [ ]:
for df_name, df in data.items():
    print(f'\n=== Informasi untuk DataFrame {df_name} ===')
    print(df.info())

Menampilkan kolom dan tipe data apa saja yang terisi di dalam data **day** dan **hour**

---

In [ ]:
columns = ['season', 'mnth', 'holiday', 'weekday', 'weathersit']

for column in columns:
    day[column] =  day[column].astype("category")
    hour[column] =  hour[column].astype("category")

Yang saya ubah disini adalah kolom *season, mnth, holiday, weekday, weathersit* dari *int* menjadi *category* karena kolom ini akan saya gunakan untuk analisis dan menjawab pertanyaan dan memiliki data dengan beberapa varian

---

In [ ]:
day.describe(include=['category'])

Mendeskripsikan data **day** untuk *category*

---

In [ ]:
hour.describe(include=['category'])

Mendeskripsikan data **hour** untuk *category*

---

In [ ]:
day['dteday'] = pd.to_datetime(day['dteday'])
hour['dteday'] = pd.to_datetime(hour['dteday'])

In [ ]:
print("Tipe data kolom 'dteday' di DataFrame day :", day['dteday'].dtype)
print("Tipe data kolom 'dteday' di DataFrame hour :", hour['dteday'].dtype)

Saya merubah kolom dteday yang tadinya *object* menjadi *datetime*

---

In [ ]:
print(day['dteday'].describe())
print(hour['dteday'].describe())

Ini adalah hasil perubahan *object* menjadi *datetime* supaya lebih mudah digunakan untuk manipulasi waktu (seperti ekstraksi hari, bulan, tahun, dll.).

---

In [ ]:
for df_name, df in data.items():
    print(f'\n~~~ Informasi untuk DataFrame {df_name} ~~~')
    print(df.info())

Ini adalah Informasi terkini terkait dataframe apa saja yang ada di data saya

---

1.3.2 Rename

Saya akan merubah nama pada kolom di data **day** dan **hour** agar lebih mudah untuk dibaca

In [ ]:
day.rename(columns={'yr':'year','mnth':'month','weekday':'one_of_week', 'weathersit':'weather_situation', 'windspeed':'wind_speed','cnt':'count_cr','hum':'humidity'},inplace=True)

In [ ]:
hour.rename(columns={'yr':'year','hr':'hours','mnth':'month','weekday':'one_of_week', 'weathersit':'weather_situation','windspeed':'wind_speed','cnt':'count_cr','hum':'humidity'},inplace=True)

In [ ]:
for df_name, df in data.items():
    print(f'\n=== Kolom-kolom dalam {df_name} ===')
    print(df.columns.tolist())

Ini adalah hasil dari perubahan nama kolom yang saya implementasikan, agar lebih mudah untuk pembacaan dan pengelolaan

---

In [ ]:
categorical_columns = ['season', 'month', 'weather_situation', 'one_of_week', 'year']

mappings = {
    'season': {1: 'Musim Semi', 2: 'Musim Panas', 3: 'Musim Gugur', 4: 'Musim Dingin'},
    'month': {1: 'Januari', 2: 'Febuari', 3: 'Maret', 4: 'April', 5: 'Mei', 6: 'Juni',
              7: 'Juli', 8: 'Agustus', 9: 'September', 10: 'Oktober', 11: 'November', 12: 'Desember'},
    'weather_situation': {1: 'Cerah', 2: 'Mendung', 3: 'Hujan Ringan', 4: 'Hujan Lebat'},
    'one_of_week': {0: 'Minggu', 1: 'Senin', 2: 'Selasa', 3: 'Rabu',
                    4: 'Kamis', 5: 'Jumat', 6: 'Sabtu'},
    'year': {0: '2011', 1: '2012'}
}

data = {'day': day, 'hour': hour}

for name, df in data.items():
    # Aplikasikan mapping untuk setiap kolom
    for col, mapping in mappings.items():
        if col in df.columns:
            # Gunakan map untuk mengganti nilai dengan deskripsi
            df[col] = df[col].map(mapping)

    if 'humidity' in df.columns:
        df['humidity'] = df['humidity'] * 100

Mengganti penamaan nilai menjadi label deskriptif agar lebih dipahami sesuai mapping yang saya tentukan, lalu mengubah juga skala kolom humidity menjadi persentase agar data lebih mudah dipahami.

---

In [ ]:
day

Menampilkan data **day**

---

In [ ]:
hour

Menampilkan data **hour**

---

In [ ]:
one_of_week = ["Senin", "Selasa", "Rabu", "Kamis", "Jumat", "Sabtu", "Minggu"]
def get_category_days(one_of_week):
    if one_of_week in ["Sabtu", "Minggu"]:
        return "weekend"
    else:
        return "weekdays"

hour["category_days"] = hour["one_of_week"].apply(get_category_days)
day["category_days"] = day["one_of_week"].apply(get_category_days)

Saya membuat kolom *category_days* untuk mengklasifikasikan setiap hari dalam kolom one_of_week sebagai *weekdays* atau *weekend* , untuk memudahkan penganalisisan terkait perbandingan di *weekdays* dan *weekend*

---

In [ ]:
week_days_df = pd.DataFrame({
    'one_of_week': one_of_week
})

week_days_df['category_days'] = week_days_df['one_of_week'].apply(get_category_days)

print(week_days_df)

Ini adalah output dari pengkategorian *weekdays* dan *weekend*

---

In [ ]:
def classify_humidity(humidity):
    if humidity < 45:
        return "Kering"
    elif humidity >= 45 and humidity < 65:
        return "Ideal"
    else:
        return "Lembab"

hour["humidity_category"] = hour["humidity"].apply(classify_humidity)
day["humidity_category"] = day["humidity"].apply(classify_humidity)

In [ ]:
print(hour[['humidity', 'humidity_category']].head())
print(day[['humidity', 'humidity_category']].head())

Disini saya menambahkan kolom baru yaitu *humidity_category* pada data **day** dan **hour** untuk mengklasifikasikan nilai kelembaban ke dalam tiga kategori: *Terlalu kering*, *Ideal*, dan *Terlalu Lembab*, berdasarkan nilai kelembaban pada kolom humidity

---

**Insight:**
- Merubah tipe data pada kolom *dteday* menjadi *datetime*
- Me-Rename kolom *yr : year, mnth : month, weekday : one_of_week, weathersit : weather_situation, windspeed : wind_speed, cnt : count_cr, hum : humidity* pada data **day** agar lebih mudah untuk dibaca
- Merubah tipe data *int* pada *season, month, holiday, weekday, workingday, weathersit* menjadi tipe data *category*
- Membuat kolom baru *category_days* (berisikan informasi *weekdays* dan *weekend*) dan *humidity_category* (berisikan nilai kelembapan *Terlalu Lembab, Ideal,* dan *Terlalu Kering*)

## **2. Exploratory Data Analysis (EDA)**

### 2.1 Explore data hour

In [ ]:
data["hour"].sample(5)

Saya menampilkan sample 5 data **hour** untuk memudahkan saya dalam membaca kolom


---

In [ ]:
result = hour.groupby(by="hours", observed=True).agg({
    "count_cr": "sum"
}).sort_values(by="count_cr", ascending=False)

print(result)

Disini saya men-sort data **hour**, dimana disini saya menghitung total penyewaan sepeda dalam rentang pukul 00.00 hingga 23.59 dari kolom *hours*, dan yang terbanyak yaitu pukul 17.00 dan tersedikit yaitu pukul 04.00

---

In [ ]:
result = hour.groupby(by="one_of_week", observed=True).agg({
    "count_cr": "sum"
}).sort_values(by="count_cr", ascending=False)

print(result)

Disini saya men-sort data **hour**, dimana disini saya menghitung total penyewaan *one_of_week* atau hari mana yang paling banyak penyewaan sepeda dan hari mana yang paling sedikit penyewaan sepeda

---

### 2.2 Explore data day

In [ ]:
data["day"].sample(5)

Saya menampilkan sample 5 data **day** untuk memudahkan saya dalam membaca kolom


---

In [ ]:
result = day.groupby(by="season", observed=True).agg({
    "count_cr": "sum"
}).sort_values(by="count_cr", ascending=False)

print(result)

Disini saya men-sort data **day**, dimana disini saya menghitung total penyewaan disetiap musimnya *season*, dan mengurutkannya dari musim paling banyak penyewaan yaitu musim gugur hingga musim paling sedikit penyewaan yaitu musim semi


---

In [ ]:
result = hour.groupby("year").agg({
    "registered": "sum",
    "casual": "sum"
})

print(result)

Disini saya menelusuri bahwasanya perbandingan seseorang peminjam yang teregistrasi menjadi member dan yang belum menjadi member *casual*, terpaku cukup jauh gap-nya, dimana bisa kita lihat di tahun 2011 member yang teregistrasi yaitu 995,851 dan yang belum menjadi member 247,252 orang

---

In [ ]:
result = day.groupby(by="month", observed=True).agg({
    "count_cr": "sum"
}).sort_values(by="count_cr", ascending=False)

print(result)

**Insight:**
- Menghitung jumlah jam dalam penyewaan sepedah terbanyak dan tersedikit pada pukul berapa dari data **hour**
- Menghitung jumlah total hari apa yang paling banyak penyewaan sepedanya dari data **hour**
- Menghitung jumlah total musim apa yang paling banyak penyewaan sepedanya dari data **day**
- Menghitung jumlah total member mana yang teregistrasi dan yang belum atau *casual* pada tahun 2011 dan 2012

## **3. Visualization & Explanatory Analysis**

### Pertanyaan 1: Bagaimana faktor cuaca (weathersit) memengaruhi jumlah penyewaan sepeda harian?

In [ ]:
# Pastikan `grouped_day` sudah didefinisikan sebelumnya
grouped_day = day.groupby(by="season", observed=True).agg({
    "count_cr": "sum"
}).sort_values(by="count_cr", ascending=False)

In [ ]:
grouped_day.head()

In [ ]:
# Define a custom color palette
colors = sns.color_palette(["#66BB6A", "#FFEB3B", "#FF7043", "#80D6FF"], n_colors=len(grouped_day))

# Visualisasi dengan data yang sudah dikelompokkan
fig, ax = plt.subplots(figsize=(35, 10))

sns.barplot(
    y=grouped_day.index,
    x=grouped_day["count_cr"],
    palette=colors,
    hue=grouped_day.index,
    ax=ax,
    order=grouped_day.index  # Menambahkan urutan berdasarkan index
)

ax.set_title("Bar Chart Antar-Musim", loc="center", fontsize=50)
ax.set_ylabel('season', fontsize=30)
ax.set_xlabel('count_cr', fontsize=30)
ax.tick_params(axis='x', labelsize=30)
ax.tick_params(axis='y', labelsize=30)

plt.show()

Dari visualisasi ini menunjukkan bahwa musim dengan penyewaan terbanyak adalah *Musim Gugur* dengan jumlah 1,061,129, sedangkan yang paling sedikit atau jarang adanya penyewaan sepeda adalah *Musim Semi* dengan Jumlah 471,348

---

### Pertanyaan 2: Apakah ada perbedaan jumlah penyewaan sepeda antara hari kerja (workingday) dan akhir pekan/libur?

In [ ]:
sum_order_items_df = hour.groupby("hours").count_cr.sum().sort_values(ascending=False).reset_index()

In [ ]:
hour_count = sum_order_items_df.groupby('hours')['count_cr'].sum().reset_index()

In [ ]:
hour_count

In [ ]:
# Mengurutkan berdasarkan penyewa terbanyak dan sedikit
top_rentals = hour_count.sort_values(by="count_cr", ascending=False).head(5)
least_rentals = hour_count.sort_values(by="count_cr", ascending=True).head(5)

# Mengatur warna untuk top dan least rentals
top_colors = ["#D3D3D3", "#D3D3D3", "#074799", "#D3D3D3", "#D3D3D3"]
least_colors = ["#D3D3D3", "#D3D3D3", "#D3D3D3", "#F26B0F", "#D3D3D3"]

# Membuat plot dengan dua sub-plot
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(30, 15))

# Plot untuk jam dengan penyewa terbanyak
sns.barplot(
    x="hours",
    y="count_cr",
    data=top_rentals,
    hue="hours",
    palette=top_colors,
    ax=ax[0]
)
ax[0].set_ylabel("Jumlah Penyewaan Sepeda", fontsize=30)
ax[0].set_xlabel("Jam Malam (PM)", fontsize=30)
ax[0].set_title("Jam dengan Penyewa Sepeda Terbanyak", loc="center", fontsize=30)
ax[0].tick_params(axis='x', labelsize=30)
ax[0].tick_params(axis='y', labelsize=30)
ax[0].legend(title="Hour", title_fontsize=20, fontsize=15)

# Plot untuk jam dengan penyewa paling sedikit
sns.barplot(
    x="hours",
    y="count_cr",
    data=least_rentals,
    hue="hours",
    palette=least_colors,
    ax=ax[1]
)
ax[1].set_ylabel("Jumlah Penyewaan Sepeda", fontsize=30)
ax[1].set_xlabel("Jam Pagi (AM)", fontsize=30)
ax[1].set_title("Jam dengan Penyewa Sepeda Tersedikit", loc="center", fontsize=30)
ax[1].invert_xaxis()  # Invert the x-axis for a better view of AM times
ax[1].tick_params(axis='x', labelsize=30)
ax[1].tick_params(axis='y', labelsize=30)
ax[1].legend(title="Hour", title_fontsize=20, fontsize=15)

# Tampilkan plot
plt.tight_layout()
plt.show()

Berdasarkan visualisasi yang saya hasilkan ini menunjukkan bahwa penyewaan sepeda itu paling sering digunakan pada waktu pukul 17:00 dengan jumlah 336,860, sedangkan waktu penyewaan paling sedikit adalah pada pukul 04:00 dengan Jumlah 4,428

---


### Pertanyaan 3: Bagaimana Perbandingan Jumlah Pengguna Casual vs Pengguna Teregistrasi?

In [ ]:
day['month'] = pd.to_datetime(day['dteday']).dt.month
day['month_name'] = day['month'].map(mappings['month'])

day['month_name'] = pd.Categorical(day['month_name'], categories=mappings['month'].values(), ordered=True)

monthly_revenue = day.groupby('month_name', observed=True)['count_cr'].sum()

In [ ]:
monthly_revenue

In [ ]:
colors = ['tab:blue' if month != 'Agustus' else 'red' for month in monthly_revenue.index]

# Membuat bar chart dengan ukuran yang lebih proporsional
plt.figure(figsize=(12, 6))  # Lebar dan tinggi lebih proporsional untuk Streamlit
bars = plt.bar(monthly_revenue.index, monthly_revenue.values, color=colors)

# Menambahkan legend untuk warna
red_patch = plt.Line2D([0], [0], color='red', lw=4, label='Bulan Sewa Terbanyak')
blue_patch = plt.Line2D([0], [0], color='tab:blue', lw=4, label='Bulan Lainnya')
plt.legend(handles=[red_patch, blue_patch], loc='upper right', fontsize=8)

# Menambahkan judul dan label
plt.title('Total Penyewaan Sepeda per Bulan', fontsize=16, pad=20)
plt.xlabel('Bulan', fontsize=14, labelpad=10)
plt.ylabel('Total Penyewaan', fontsize=14, labelpad=10)
plt.xticks(rotation=45, fontsize=12)  # Agar label bulan lebih jelas
plt.yticks(fontsize=12)

# Menambahkan nilai di atas setiap bar
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2, height + 5,  # Posisi teks
        f'{int(height)}',  # Menampilkan nilai dalam format integer
        ha='center', va='bottom', fontsize=10
    )

# Menampilkan grafik
plt.tight_layout()
plt.show()

Bisa kita lihat bahwa Penyewaan / Penyewaan sepeda terbanyak dalam kategori bulanan itu adalah bulan Agustus dengan jumlah 351,194

## **4. Analisis Lanjutan**

## 4.1 RFM Analysis

Dalam Penyewaan Sepeda, kita dapat mengadaptasi konsep RFM untuk:

*   Recency: Hari terakhir pengguna meminjam sepeda, semakin kecil *recency* menandakan baru bertransaksi
*   Frequency: Jumlah total penyewaan sepeda oleh pengguna.
*   Monetary: Total durasi atau jumlah penyewaan sepeda, semakin besar *monetary* menandakan pengeluarannya untuk meminjam sepeda besar

In [ ]:
day['last_purchase_date'] = pd.to_datetime(day['dteday'])
current_date = day['last_purchase_date'].max()
day['recency'] = (current_date - day['last_purchase_date']).dt.days

frequency = day.groupby('instant').size().reset_index(name='frequency')
monetary = day.groupby('instant')['count_cr'].sum().reset_index(name='monetary')

rfm = pd.merge(frequency, monetary, on='instant')

rfm['recency'] = day.groupby('instant')['recency'].min().reset_index(drop=True)

rfm['customer_segment'] = rfm.apply(
    lambda x: 'Registered' if x['frequency'] > 5 or x['monetary'] > 1000 else 'Casual', axis=1
)

print(rfm)

Disini saya menampilkan range dari data 0 hingga 730, bisa kita lihat contohnya transaksi terbaru itu dilakukan oleh data ke 730 atau *recency* = 0 dengan *monetary* sebesar 2729 dan Tipe Pengguna yang sudah teregistrasi

---

In [ ]:
top_5_monetary = rfm.sort_values(by='monetary', ascending=False).head(5)

print(top_5_monetary)

Disini saya menampilkan *monetary* terbesar atau terbanyak itu adalah sebesar 8714 oleh pengguna dengan *recency* = 107 dan Tipe Pengguna yang sudah Teregistrasi

---

## 4.2 Clusterring

### 4.2.1 Clusterring Pengguna dari data RFM

In [ ]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['recency', 'frequency', 'monetary']])

wcss = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o')
plt.title('Metode Siku untuk Jumlah Kluster yang Optimal')
plt.xlabel('Number of Clusters')
plt.ylabel('WCSS')
plt.show()

optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42, n_init=10)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

In [ ]:
rfm

Cluster di atas dapat diartikan sebagai berikut:


*   Cluster 0 = Pengguna yang cenderung melakukan transaksi dengan jumlah *Monetary* yang menengah. Mereka cenderung memiliki *Recency* yang lebih tinggi, menunjukkan bahwa mereka telah melakukan transaksi lebih lama.
*   Cluster 1 = Pengguna dengan *Monetary* yang lebih tinggi dan Recency yang lebih rendah. Ini adalah pengguna yang relatif baru dan melakukan transaksi dengan jumlah uang yang besar.
*   Cluster 2 = Pengguna dengan *Monetary* yang lebih rendah dan Recency yang lebih tinggi. Mereka adalah pengguna yang sudah lama melakukan transaksi tetapi dengan jumlah yang lebih kecil.

---


### 4.2.2 Clusterring Manual Group Humidity

In [ ]:
result = hour.groupby(by="humidity_category").agg({"count_cr": ["count"]})

# Sort the results in descending order
sorted_result = result.sort_values(by=('count_cr', 'count'), ascending=False)

print(sorted_result)

BIsa kita lihat dari data **hour** bahwa pengguna akan lebih sering meminjam di keadaan *humidity* Lembab dibandingkan *Humidity* Kering

---

### 4.2.3 Clusterring Manual Group Weather Situation

In [ ]:
count_weather = hour.groupby(by="weather_situation", observed=False).size().sort_values(ascending=False)

print(count_weather)

Bisa kita lihat dari data **hour** bahwa pengguna penyewaan sepedah lebih cenderung meminjam sepeda ketika cuacanya itu *Cerah* dengan 11413, dan ada juga pengguna yang meminjam ketika *Hujan Lebat* = 3


---

## Conclusion

- Conclution Pertanyaan 1 : Bagaimana faktor cuaca (weathersit) memengaruhi jumlah penyewaan sepeda harian?
- Conclution Pertanyaan 2 : Apakah ada perbedaan jumlah penyewaan sepeda antara hari kerja (workingday) dan akhir pekan/libur?
- Conclution Pertanyaan 3 : Berapa banyak Total Penyewaan Setiap Disetiap Bulannya?

- Pertanyaan 1 : Berdasarkan visualisasi yang saya berikan itu sudah menjawab pertanyaan, dimana saya menampilkan penyewaan antar musim dan memplotkannya dalam kategori 4 musim yang dimana *Musim Gugur* adalah musim yang paling sering untuk orang menyewa sepeda. Hal ini bisa didasarkan oleh cuacanya yang sejuk dan nyaman, dan pemandangan alamnya yang indah nan menawan.
- Pertanyaan 2 : Berdasarkan visualisasi yang saya berikan itu sudah sangat menjawab pertanyaan, dimana saya menampilkan penyewaan dalam range terbanyak dan tersedikit, yang dimana pada pukul 17:00 adalah pukul terbanyak orang menyewa sepeda dan pada pukul 04:00 adalah paling sedikit orang menyewa sepeda, yang dapat kita simpulkan pada range pukul 01:00 hingga 05:00 adalah waktu Istirahat dan pada pukul 17:00 adalah waktu untuk pulang bekerja.
- Pertanyaan 3 : Berdasarkan visualisasi yang saya berikan itu sudah sangat menjawab pertanyaan, dimana saya menampilkan jumlah total penyewaan setiap bulannya dan menentukan juga bulan dengan penyewaan terbanyak yaitu bulan Agustus dengan jumlah penyewaan 351,194


In [ ]:
day.to_csv('main_data_day.csv', index=False)

In [ ]:
hour.to_csv('main_data_hour.csv', index=False)